# Project: Pressure sensor placement for leakage detection

### Introduction

(This notebook is based on the epyt-flow tutorial for leakage detection found here: [Tutorials](https://github.com/WaterFutures/EPyT-and-EPyT-Flow-Tutorial))

**Authors:** Timo Beckmann, Tobias Kroecker, Yasmine Marzouk

This notebook documents a research project focused on evaluating two critical aspects of leakage detection in water distribution networks:

1. **Sensor Placement Strategy**: We implement and evaluate a sensor placement algorithm based on information theory. This approach aims to optimize the placement of pressure sensors by maximizing relevance and minimizing redundancy, ensuring effective network coverage. The evaluation includes varying the number of sensors to analyze the impact of sensor density on detection performance.

2. **Leakage Detection Method**: A machine learning-based leakage detection approach is implemented using a bagging classifier with decision trees. This method leverages ensemble learning to enhance robustness and accuracy in detecting anomalies such as leakages in the network.

3. **Datasets**: 
    - **LeakDB**: A benchmark dataset providing realistic scenarios for leakage detection in water distribution networks, used to evaluate both the sensor placement strategy and the leakage detection method.
    - **BattLeDIM**: A dataset offering realistic demand patterns and scenarios, used to simulate and validate the sensor placement strategy.

4. **Evaluation Goals**: The primary objective is to assess the performance of the leakage detection method and the effectiveness of the sensor placement strategy. This includes:
    - Analyzing the detection accuracy of the machine learning model.
    - Investigating the influence of different sensor placement sizes on detection performance.

By combining theoretical insights from information theory and machine learning with practical applications, this notebook provides a comprehensive evaluation of the interplay between sensor placement and leakage detection in water distribution networks.

## Imports and method definitions

### Imports

In [11]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=ImportWarning)
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
import matplotlib.pyplot as plt
import sklearn
import epyt_flow
import pandas as pd
import random

from sklearn.metrics import mutual_info_score
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from epyt_flow.data.networks import load_ltown
from epyt_flow.data.benchmarks import load_leakdb_scenarios
from epyt_flow.simulation import ScenarioSimulator
from epyt_flow.simulation.events import AbruptLeakage, IncipientLeakage
from epyt_flow.utils import to_seconds, time_points_to_one_hot_encoding
from epyt_control.signal_processing import SensorInterpolationDetector
from joblib import Parallel, delayed, parallel_backend

### Implementation of the sensor placement algorithm

In [12]:
# Assume that we know which specific nodes we want to watch:
def calc_Relevance (node_pressures, relevant_index):
    relevances = {}
    for i in range(len(node_pressures)):
        relevances[i] = mutual_info_score(node_pressures[relevant_index], node_pressures[i])
    return relevances

# calcuate relevance over all nodes:

def calc_Relevances (node_pressures):
    relevances = {}
    for i in range(len(node_pressures)):
        for j in range(i + 1, len(node_pressures)):
            m_i_score = mutual_info_score(node_pressures[i], node_pressures[j])
            if i not in relevances:
                relevances[i] = m_i_score
            else:
                relevances[i] += m_i_score
            if j not in relevances:
                relevances[j] = m_i_score
            else:
                relevances[j] += m_i_score
    return relevances
    
def calc_Redundance (x_pressures, sensor_pressures):
    redundancy = 0
    for pressure in sensor_pressures:
        redundancy += mutual_info_score(x_pressures, pressure)
    redundancy /= len(sensor_pressures)
    return redundancy

In [13]:
def calc_Relevances_parallel(node_pressures, n_jobs=1):
    relevances = Parallel(n_jobs=n_jobs)(delayed(calc_Relevance)(node_pressures, i) for i in range(len(node_pressures)))
    return {i: sum(relevance[i] for relevance in relevances) for i in range(len(node_pressures))}

In [14]:
def calc_sensor_placement(node_pressures, num_sensors = 15): # do we know which nodes we want to monitor? junctions? all of them?
    remaining_nodes = list(range(len(node_pressures)))
    relevances = calc_Relevances(node_pressures)  # Assume we want to know the relevances in regard to all nodes
    Sensor_placement = [max(relevances, key=relevances.get)]
    remaining_nodes.remove(Sensor_placement[-1])
    sensor_pressures = [node_pressures[Sensor_placement[-1]]]
    no_redundance_nodes = [a for a in remaining_nodes if calc_Redundance(node_pressures[a], node_pressures[Sensor_placement]) == 0]
    while no_redundance_nodes: # add nodes of highest relevance without any redundance
        if (len(Sensor_placement) >= num_sensors):
            break
        tmp_relevances = relevances.copy()
        for i in range(len(tmp_relevances.keys())): # remove nodes that have been added to the sensors and nodes with redundance
            if i in Sensor_placement:
                del tmp_relevances[i]
                continue
            if i not in no_redundance_nodes:
                del tmp_relevances[i]
        Sensor_placement.append(max(tmp_relevances, key=relevances.get))
        remaining_nodes.remove(Sensor_placement[-1])
        sensor_pressures.append(node_pressures[Sensor_placement[-1]])
        no_redundance_nodes = [a for a in remaining_nodes if calc_Redundance(node_pressures[a], sensor_pressures) == 0]
    remaining_relevances = list(map(relevances.get, remaining_nodes))
    while not(all(list(map(lambda x: x == 0, remaining_relevances)))) and remaining_nodes:
        if (len(Sensor_placement) >= num_sensors):
            break
        RRI = {}
        for i in remaining_nodes:
            RRI[i] = relevances[i]/ calc_Redundance(node_pressures[i], sensor_pressures)
        Sensor_placement.append(max(RRI, key=RRI.get))
        sensor_pressures.append(node_pressures[Sensor_placement[-1]])
        remaining_nodes.remove(Sensor_placement[-1])
        remaining_relevances = list(map(relevances.get, remaining_nodes))
    return Sensor_placement

In [15]:
def calc_sensor_placement_parallel(node_pressures, num_sensors = 15, n_jobs = 1): # do we know which nodes we want to monitor? junctions? all of them?
    remaining_nodes = list(range(len(node_pressures)))
    if n_jobs > 1:
        relevances = calc_Relevances_parallel(node_pressures, n_jobs=n_jobs)
    else:
        relevances = calc_Relevances(node_pressures)  # Assume we want to know the relevances in regard to all nodes
    Sensor_placement = [max(relevances, key=relevances.get)]
    remaining_nodes.remove(Sensor_placement[-1])
    sensor_pressures = [node_pressures[Sensor_placement[-1]]]
    Redundances = Parallel(n_jobs=n_jobs)(delayed(calc_Redundance)(node_pressures[a], node_pressures[Sensor_placement]) for a in remaining_nodes)
    no_redundance_nodes = [remaining_nodes[a] for a in range(len(remaining_nodes)) if Redundances[a] == 0]
    while no_redundance_nodes: # add nodes of highest relevance without any redundance
        if (len(Sensor_placement) >= num_sensors):
            break
        tmp_relevances = relevances.copy()
        for i in range(len(tmp_relevances.keys())): # remove nodes that have been added to the sensors and nodes with redundance
            if i in Sensor_placement:
                del tmp_relevances[i]
                continue
            if i not in no_redundance_nodes:
                del tmp_relevances[i]
        Sensor_placement.append(max(tmp_relevances, key=relevances.get))
        remaining_nodes.remove(Sensor_placement[-1])
        sensor_pressures.append(node_pressures[Sensor_placement[-1]])
        Redundances = Parallel(n_jobs=n_jobs)(delayed(calc_Redundance)(node_pressures[a], node_pressures[Sensor_placement]) for a in remaining_nodes)
        no_redundance_nodes = [remaining_nodes[a] for a in range(len(remaining_nodes)) if Redundances[a] == 0]
    remaining_relevances = list(map(relevances.get, remaining_nodes))
    while not(all(list(map(lambda x: x == 0, remaining_relevances)))) and remaining_nodes:
        if (len(Sensor_placement) >= num_sensors):
            break
        RRI = {}
        for i in remaining_nodes:
            RRI[i] = relevances[i]/ calc_Redundance(node_pressures[i], sensor_pressures)
        Sensor_placement.append(max(RRI, key=RRI.get))
        sensor_pressures.append(node_pressures[Sensor_placement[-1]])
        remaining_nodes.remove(Sensor_placement[-1])
        remaining_relevances = list(map(relevances.get, remaining_nodes))
    return Sensor_placement

### Implementation of the bagging classifier

In [16]:
# DecisionTreeClassifier with entropy criterion is in the ID3 style. TODO: Might use decision-tree-id3 package if it works?
def mixed_model_classification_fit(n_estimators, X_train, y_train, n_jobs=1):
    with parallel_backend('threading', n_jobs=n_jobs):
        base = DecisionTreeClassifier(criterion='entropy') #Entropy criterion should be approximate to ID3 decision tree. Sadly no official ID3 Implementation is given
        bagged = BaggingClassifier(estimator=base, n_estimators=n_estimators, max_samples=0.8, oob_score=True) #for faster runtime adjust parameters, especially n_estimators
        bagged.fit(X_train, y_train)
    return bagged

def mixed_model_classification_predict(classifier, X_test, y_test, n_jobs=1):
    with parallel_backend('threading', n_jobs=n_jobs):
        # Using the classifier to predict the labels for the test set
        # This will also print the classification report
        y_pred = classifier.predict(X_test)
    print(classification_report(y_test, y_pred, labels=np.unique(y_test),
                                target_names=[f"class_{int(c)}" for c in np.unique(y_test)], output_dict=False))
    return y_pred


In [ ]:
# test the classifier:
data_dict = epyt_flow.data.benchmarks.leakdb.load_data(
    scenarios_id=["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"],  
    #scenarios_id=["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"],
    use_net1=False,
    return_X_y=True
)

X_list, y_list = [], [] 
for sid, (Xi, yi) in data_dict.items():
    X_list.append(Xi.values if hasattr(Xi, "values") else Xi) #Check for correctness. 
    y_list.append(yi)

X = np.vstack(X_list)        # shape → (total_samples, n_features)
y = np.concatenate(y_list)   # shape → (total_samples,)

print(f"Total samples: {X.shape[0]}, features: {X.shape[1]}")

X_train, X_test, y_train, y_test = train_test_split( #Might use datasets given train-test split
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

clf = mixed_model_classification_fit(100, X_train, y_train)

In [ ]:
mixed_model_classification_predict(clf, X_test, y_test)

## LeakDB

###  Loading scenarios and placing sensors

In this part a subset of the scenarios from LeakDB is loaded and based on existiong pressure data from the different scenarios, the sensor placements are calculated. The scenarios are split into a train and a testset, based on their ids. The pressure sensors are placed based on the training data, assuming that in a realistic scenario the sensors could only be placed based on a finite set of pressure data.

In [ ]:
#TODO: adapt to possibility of using battledim
#NOTE: this approach calculates a sensor placement based on multiple scenarios, i.e. all scenarios we look at
#      due to the complexity of the algorithm this will take a lot of time!
scen_train_keys = [str(num) for num in random.sample(range(1, 501), 10)]
scen_test_keys = ["511", "512", "513", "514", "515", "516", "517", "518", "519", "520"]
scen_keys = scen_train_keys + scen_test_keys
data = epyt_flow.data.benchmarks.leakdb.load_data(scenarios_id=scen_train_keys, use_net1=False, return_X_y=False)

pressure_column_heads = data[scen_train_keys[0]].columns[:32]
node_pressures = []
for key in scen_train_keys:
    for i in range(len(pressure_column_heads)):
        if key == scen_train_keys[0]:
            node_pressures.append(data[key]['Pressure-Node_' + str(i + 1)])
        else:
            node_pressures[i] = np.concatenate((node_pressures[i], data[key]['Pressure-Node_' + str(i + 1)]))
node_pressures = np.array(node_pressures)
node_pressures.shape

In [ ]:
parallel_sensor_placements = calc_sensor_placement_parallel(node_pressures, num_sensors=15, n_jobs=4)
sensor_placements = calc_sensor_placement(node_pressures, num_sensors=15)
print("Parallel sensor placements:", parallel_sensor_placements)
print("Sensor placements:", sensor_placements)

In [9]:
configs = load_leakdb_scenarios(scenarios_id=scen_keys, use_net1=False, verbose=True)
scenarios = {}
for i, key in enumerate(scen_keys):
    scenarios[key] = ScenarioSimulator(scenario_config=configs[i])

The topology of the water distribution network does not change between the different scenarios. Here the general topology is inspected.

In [ ]:
scenarios[scen_train_keys[0]].plot_topology()

Run the complete set of simulations:

In [ ]:
scada_data = {}
for key, scenario in scenarios.items():
    scada_data[key] = scenario.run_simulation(verbose=True)

### Machine Learning based Leakage Detection

Prepare the simulation results for calibrating (i.e. creating) a Machine Learning based leakage detection method:

- Create a feature vector (pressure readings at the sensors).
- Create ground-truth labels utilizing the [`time_points_to_one_hot_encoding()`](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.html#epyt_flow.utils.time_points_to_one_hot_encoding) helper function.

The scenarios are already split into a train and test set. Because of that, two feature vectors and lable-sets are created from the start, one of each for the training of the classifier and another pair for the testing.

In [ ]:
# Concatenate pressure and flow readings into a single feature vector
# X = np.concatenate((scada_data.get_data_pressures(), scada_data.get_data_flows()), axis=1)
"""X_tmp = [scada_data[key].get_data_pressures()[:, sensor_placements] for key in scen_keys]
X = np.vstack(X_tmp)
y_tmp = []
for key in scen_keys:
    events_times = [int(t / configs[int(key)].general_params["hydraulic_time_step"])
                    for t in scenarios[key].get_events_active_time_points()]
    y_tmp.append(time_points_to_one_hot_encoding(events_times, total_length=X_tmp[0].shape[0]))
y = np.concatenate(y_tmp)
print(f"Total samples: {X.shape[0]}, features: {X.shape[1]}, samples_y : {y.shape[0]}")"""
X_tmp = [scada_data[key].get_data_pressures()[:, sensor_placements] for key in scen_train_keys]
X_train = np.vstack(X_tmp)
X_test = {key: scada_data[key].get_data_pressures()[:, sensor_placements] for key in scen_test_keys}
y_tmp = []
y_test = {}
for i, key in enumerate(scen_train_keys):
    events_times = [int(t / configs[i].general_params["hydraulic_time_step"])
                    for t in scenarios[key].get_events_active_time_points()]
    y_tmp.append(time_points_to_one_hot_encoding(events_times, total_length=X_tmp[0].shape[0]))
for i, key in enumerate(scen_test_keys):
    events_times = [int(t / configs[i].general_params["hydraulic_time_step"])
                    for t in scenarios[key].get_events_active_time_points()]
    y_test[key] = time_points_to_one_hot_encoding(events_times, total_length=X_test[key].shape[0])
y_train = np.concatenate(y_tmp)
print(f"Total samples: {X_train.shape[0]}, features: {X_train.shape[1]}, samples_y : {y_train.shape[0]}")

The training data is shuffled to avoid biases: 

In [ ]:
"""X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)"""


X_train, y_train = sklearn.utils.shuffle(
    X_train, y_train,
    random_state=42
)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

In [ ]:
mixed_clf = mixed_model_classification_fit(100, X_train, y_train)

Apply the classifier to the test data (i.e. the test scenarios). Note that one-hot-encoding isn't necessary here, since our classifier is already trained to return one prediction, 0 or 1, per timestep:

In [ ]:
suspicious_time_points = {key: mixed_model_classification_predict(mixed_clf, X_test[key], y_test[key]) for key in scen_test_keys}
#suspicious_time_points_train = {key: mixed_model_classification_predict(mixed_clf, X_tmp[i], y_tmp[i]) for i, key in enumerate(scen_train_keys)}
#suspicious_time_points = {i: mixed_model_classification_predict(mixed_clf, X_tmp[i], y_tmp[i]) for i in range(len(X_tmp))} 

### Evaluation

In order to evaluate the performance of the leakage detector, we compute a confusion matrix, plot the leaks as well as the alarms raised by the classifier and apply a few other metrics (as seen with the predictions).

Here, we plot event (i.e. leakage) presence over time together with the raised alarms by the detector:

In [ ]:
# Generate plots for test scenarios
for key in scen_test_keys:
    plt.figure()
    plt.plot(list(range(len(y_test[key]))), y_test[key], color="red", label="Ground truth")
    plt.bar(list(range(len(suspicious_time_points[key]))), suspicious_time_points[key], label="Raised alarm")
    plt.title(f"Test Scenario {key}")
    plt.legend()
    plt.ylabel("Leakage indicator")
    plt.yticks([0, 1], ["Inactive", "Active"])
    plt.xlabel("Time (5min steps)")
    plt.show()

"""# Generate plots for training scenarios
for i, key in enumerate(scen_train_keys):
    plt.figure()
    plt.plot(list(range(len(y_tmp[i]))), y_tmp[i], color="red", label="Ground truth")
    plt.bar(list(range(len(suspicious_time_points_train[key]))), suspicious_time_points_train[key], label="Raised alarm")
    plt.title(f"Training Scenario {key}")
    plt.legend()
    plt.ylabel("Leakage indicator")
    plt.yticks([0, 1], ["Inactive", "Active"])
    plt.xlabel("Time (5min steps)")
    plt.show()"""

In [19]:
for scenario in scenarios.values():
    scenario.close()

## BattLeDIM